# A look into creating interactive maps with folium

We'll be using the individual datasets at first, to get individual countries' maps. Once we have the work on individual countries, we'll start trying to build a european map

We start by importing the required packages

In [ ]:
import dataGathering
import countriesAndCities
import folium
import geopandas as gpd
import pandas as pd
from shapely import wkt

# 1. Austria

In [ ]:
def changeKeys(country, valueToChange, newValue):
    '''A method that takes the keys for a country in the largestStations dictionary, and replacey certain values
     @param country: the country with a value to change, of type string
     @param valueToChange: the value in the key to change
     @param newValue: the new value in the key
     @return largestStations: a dictionary with the information, of type dict'''
    listKeys = list(largestStations[country].keys())
    oldKeys = []
    for i in range (len(listKeys)):
        station = listKeys[i]
        if valueToChange in listKeys[i]:
            oldKey = station
            oldKeys.append(oldKey)
            newKey = station.replace(valueToChange, '') + newValue
            largestStations[country][newKey] = largestStations[country][oldKey]
    
    for station in oldKeys:
        largestStations[country].pop(station)
    
    return(largestStations)

We build the dataset the same way we built it for the unified database

In [ ]:
largestStations = dict()

urlGermany = 'https://bahnauskunft.info/bahnhoefe-deutschland/'
urlAustria = 'https://www.omio.at/bahnhoefe'

In [ ]:
largestStations['Austria']= dataGathering.gather(urlAustria, start=1)

In [ ]:
largestStations = changeKeys('Austria', 'Hbf', 'Hauptbahnhof')

# 1.1 Stations

In [ ]:
stationsAustria = 'Austria/GIP_Betriebsstellen_DelEUV_JSON.json'
stationsAustriaFrame = gpd.read_file(stationsAustria)

In [ ]:
columnsToRemove = ['BSTS_ID', 'DB640_CODE', 'OBJECTID', 'GIP_OBID', 'EXTERNALID', 'REGIONALCO', 'VALIDFROM', 'VALIDTO', 'OWNER_NAME', 'PV_EVA_NR', 'ANZ_AUFZUG', 'ANZ_FAHRTR', 'ANZ_UHREN',
                  'ANZ_AKUSTI','ANZ_OPTISC', 'INFOPOINT', 'MUEZ', 'MUEZ_KURZ', 'HILFE_MOBI', 'ANZ_ROLLST', 'ANZ_E_LADE', 'RUD_PARKPL', 'VERIFIZIER',
                  'PUBL_WLAN', 'MUEZ_LANG', 'BEMERKUNG']

In [ ]:
for column in columnsToRemove:
    stationsAustriaFrame = stationsAustriaFrame.drop(column, axis=1)

In [ ]:
dfStationsAustria = []
for station in largestStations['Austria']:
    tempFrame = stationsAustriaFrame.loc[stationsAustriaFrame['NAME_FPL'] == station]
    dfStationsAustria.append(tempFrame)

In [ ]:
workFrameAustria = gpd.GeoDataFrame(pd.concat(dfStationsAustria), crs = 31287)

In [ ]:
workFrameAustria = gpd.GeoDataFrame(pd.concat(dfStationsAustria), crs = 31287)

In [ ]:
workFrameAustria.head()

# 1.2 Lines

In [ ]:
linesAustria = 'Austria/GIP_Strecken_MLA.json'
linesAustriaFrame = gpd.read_file(linesAustria)

In [ ]:
uselessColumns = ['GIP_OBID', 'BST_ID', 'FOW_NAME', 'FRC_NAME', 'REGION', 'VALIDFROM', 'VALIDTO', 'CROSSSECT', 'CROSS_NAME', 
                  'ELEKTRI', 'EXPDATE']

for column in uselessColumns:
    linesAustriaFrame = linesAustriaFrame.drop(column, axis=1)

In the case of the lines, we can't remove any rows of the dataframe. Every line has a bit of information, that cannot be replicated based on other rows

# 2. Visualisation

We now have two sets of working data. We can visualise this data on interactive maps using folium

In [ ]:
trainMap = folium.Map(location = [47.5, 14.6], tiles = 'OpenStreetMap', zoom_start=12)



We can now add markers, based on the different train stations, onto the map

In [ ]:
austriaStations = [[point.xy[1][0], point.xy[0][0]] for point in workFrameAustria.geometry ]

In [ ]:
for i in range (len(austriaStations)):
    coordinates = austriaStations[i]
    station = workFrameAustria['NAME_FPL'].values[i]
    dailyVisitors = largestStations['Austria'][station][0]
    trainMap.add_child(folium.Marker(location = coordinates, popup = station + "<br>" + dailyVisitors + ' daily visitors'))

In [ ]:
trainMap

In [ ]:
for line in linesAustriaFrame.geometry:
    lineTuple = []
    lineTupleInvert = list(line[0].coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap

# 2. Germany

We can do the exact same for the German train lines

In [ ]:
largestStations['Germany'] = dataGathering.gather(urlGermany)

In [ ]:
largestStations = changeKeys('Germany', 'Hauptbahnhof', 'Hbf')

# 2.1 German Stations

In [ ]:
stations = 'Germany/railwayStationNodes.geojson'

deutscheBahnStations = gpd.read_file(stations)

In [ ]:
deutscheBahnStations = deutscheBahnStations.drop('formOfNode', axis = 1)
deutscheBahnStations = deutscheBahnStations.drop('id', axis = 1)

In [ ]:
dfListStations = []
for station in (list(largestStations['Germany'].keys())):
    tempFrame = deutscheBahnStations.loc[deutscheBahnStations['geographicalName'] == station]
    dfListStations.append(tempFrame)

In [ ]:
workFrameStationsGermany = gpd.GeoDataFrame(pd.concat(dfListStations, ignore_index=True), crs=4258)

In [ ]:
workFrameStationsGermany

In [ ]:
workFrameStationsGermany = workFrameStationsGermany.drop_duplicates(subset='railwayStationCode')

# 2.2 German train lines

In [ ]:
deutscheBahnLines = gpd.read_file('Germany/railwayLines.geojson')

# 2.3 Visualising German data

In [ ]:
germanStationsList = [[point.xy[1][0], point.xy[0][0]] for point in workFrameStationsGermany.geometry ]

In [ ]:
for i in range(len(germanStationsList)):
    coordinates = germanStationsList[i]
    station = workFrameStationsGermany['geographicalName'].values[i]
    dailyVisitors = largestStations['Germany'][station][0]
    trainMap.add_child(folium.Marker(location = coordinates, popup=station + "<br>" + dailyVisitors + ' daily visitors'))

In [ ]:
trainMap

In [ ]:
for line in deutscheBahnLines.geometry:
    lineTuple = []
    lineTupleInvert = list(line[0].coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap

The German data is awfully incomplete. We can add further information on the German rail network by using a global database

In [ ]:
globalData = 'World/wld_trs_railways_wfp.csv'
worldData = pd.read_csv(globalData)

In [ ]:
Germany = worldData[worldData['country'] == 'Germany']

In [ ]:
Germany['shape'] = Germany['shape'].apply(wkt.loads)

In [ ]:
geodata = gpd.GeoSeries(Germany['shape'])

In [ ]:
geodata.head()

In [ ]:
Germany.set_geometry('shape')

In [ ]:
for line in geodata.geometry:
    lineTuple = []
    lineTupleInvert = list(line[0].coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap

# 3 A look into the French railway network



---
A look into the French railway network 

We are using databases coming from French government open-data portal and French national railway company (SNCF) : https://www.data.gouv.fr/fr/ and https://ressources.data.sncf.com/pages/accueil/ \
We focus on two databases providing information either on the railway stations or on tne rail connections. \
The databases are : referentiel-gares-voyageurs.geojson and formes-des-lignes-du-rfn.geojson

# 3.1 French Stations

We first select the relevant cities. We focus on a dozen of them, the biggest ones. We chose : Paris, Lyon, Bordeaux, Toulouse, Brest, Marseille, Nice, Strasbourg, Rennes, Motpellier and Grenoble. \
Some of these cities have several stations. For more simplicity, there is a focus on the relevant stations connecting those cities.

In [ ]:
f_station = gpd.read_file('France/referentiel-gares-voyageurs.geojson')

In [ ]:
values = ['Paris', 'Lyon', 'Strasbourg', 'Lille', 'Bordeaux', 'Massy', 'Marseille', 'Chessy', 'Brest', 'Rennes', 'Toulouse', 'Nice', 'Montpellier', 'Grenoble']
f_station = f_station[f_station.commune_libellemin.isin(values)] 

f_station = f_station[f_station['segmentdrg_libelle']=='a']	

There is a selection on two criteria : the name of the city and the importance of the line between cities (segmentdrg_libelle). \
There is a focus on national and international railways. \


Note :  \
commune_libellemin = name of the city \
segmentdrg_libelle = a,b or c indicates the size of the railroad

In [ ]:
f_station.loc[:,'Country'] = 'France'

Additional column : name of the country and drop of the contingent.

In [ ]:
f_station.drop(['tvs', 'tvss',
       'gare_alias_libelle_fronton', 'commune_code', 'uic_code', 'rg_libelle',
       'gare', 'adresse_cp', 'code_gare', 'latitude_entreeprincipale_wgs84',
       'gare_agencegc_libelle', 'departement_libellemin', 'code',
       'gare_drg_on', 'gare_etrangere_on', 'segmentdrg_libelle',
       'gare_regionsncf_libelle', 'longitude_entreeprincipale_wgs84',
       'gare_ut_libelle', 'gare_nbpltf', 'alias_libelle_noncontraint',
       'departement_numero', 'niveauservice_libelle', 'gare_ug_libelle'],1,inplace=True)

In [ ]:
f_station

# 3.2 French railway lines

Now, we select the relevant lines between those cities

In [ ]:
lines_france = gpd.read_file('France/formes-des-lignes-du-rfn.geojson')

We only focus on the exploited lines. We drop the lines that are unexploited or private.

In [ ]:
lines_france.drop(lines_france.loc[lines_france['libelle'] != 'Exploitée'].index, inplace = True)

Selection the id of the lines of interest :

In [ ]:
values = [ '930000', '752000', '810000', '640000', '566000','956000', '957000', '991300', '752330', '893000', '431000', '420000','834000', '226000', '005000', '905000', '752340', '226310', '005390', '752100' ]

In [ ]:
lines_france = lines_france[lines_france.code_ligne.isin(values)] 

In [ ]:
lines_france.head()

# 3.3 Visualisation
And a plot of the beautiful map :

In [ ]:
franceStations = [[point.xy[1][0], point.xy[0][0]] for point in f_station.geometry ]

In [ ]:
for i in range(len(franceStations)):
    coordinates = franceStations[i]
    station = f_station['gare_alias_libelle_noncontraint'].values[i]
    #dailyVisitors = largestStations['Germany'][station][0]
    trainMap.add_child(folium.Marker(location = coordinates, popup=station))

In [ ]:
for line in lines_france.geometry:
    lineTuple = []
    lineTupleInvert = list(line.coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap


# 4 A look into the Belgian railway network

We are using databases coming from Belgian government open-data portal : https://data.gov.be/fr \
We focus on one database providing information on the railway stations and rail connections. \
The base is : tc-trajet-train-statique-sncb.geojson

# 4.1 Belgian railway stations

Because of the small size of the country, there is a focus on the cities of Brussels, Antwerp, and Gent. We also add the connection with France : line between Brussels and Lille in France, and between Antwerp and Rotterdam in the Netherland. \
Because of the poor data , we chose to create our own dataset. The very small amount of datas studied is a key element of such a choice. 
So, there is the creation of a dataframe with relevant coordinates.

In [ ]:
df = pd.DataFrame(
    {'City' : ['Brussel', 'Brussel','Gent', 'Atwerpen', 'Rotterdam',"Amsterdam"],
     'Name': ['Brussel-Zuid','Brussel-Noord','Gent-Sint-Pieters','Antwerpen-Centraal','Rotterdam-Centraal', 'Amsterdam-Centraal'],
     'Country': ['Belgium', 'Belgium', 'Belgium', 'Belgium', 'Netherland', 'Netherland'],
     'Longitude': [4.33545, 4.36015, 3.71083, 4.42115, 4.46883, 4.90032],
     'Latitude': [50.83605, 50.86045, 51.03609, 51.21722, 51.92528, 52.37913]})

Then, transformation of the dataframe to a geopandas one : the coordinates are transformed into a geometry recognized when plotted on a map.

In [ ]:
b_station = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [ ]:
b_station.drop(['Longitude', 'Latitude'],1, inplace = True)
b_station

# 4.2 Belgian lines

After selection of the stations, we matched them with their id. Then we have to clean the geodataframe and add the last missing information.

In [ ]:
lines_belgium = gpd.read_file('Belgium/tc-trajet-train-statique-sncb.geojson')

In [ ]:
values = [603,140,166,126,259,257,54,274,172,602]
lines_belgium = lines_belgium[lines_belgium.id.isin(values)] 
lines_belgium.index = range(10)

In [ ]:
departure = []
arrival = []
for k in range(10) :
  index_ = lines_belgium.long_name[k].index('--')
  departure.append(lines_belgium["long_name"][k][:index_])
  arrival.append(lines_belgium["long_name"][k][index_+3:])

lines_belgium.loc[:,"Departure"] = departure
lines_belgium.loc[:,"Arrival"] = arrival
lines_belgium.loc[:,"Country_dep"] = ['France', 'Belgium', 'Belgium', 'Belgium', 'Belgium', 'France', 'Netherland','Belgium', 'Belgium', 'Netherland']
lines_belgium.loc[:,"Country_arr"] = 'Belgium'
lines_belgium.drop(['short_name', 'long_name', 'id', 'route_type', 'color'],1,inplace=True)

In [ ]:
lines_belgium.head()

#4.3 Visualisation
And a visualisation of the achieved map :

In [ ]:
belgiumStation = [[point.xy[1][0], point.xy[0][0]] for point in b_station.geometry ]

In [ ]:
for i in range(len(belgiumStation)):
    coordinates = belgiumStation[i]
    station = b_station['Name'].values[i]
    #dailyVisitors = largestStations['Germany'][station][0]
    trainMap.add_child(folium.Marker(location = coordinates, popup=station))

In [ ]:
for line in lines_belgium.geometry:
    lineTuple = []
    lineTupleInvert = list(line[0].coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap

# 5. Switzerland railways

# 5.1 The lines

In [ ]:
import geopandas as gpd
import pandas as pd
#The lines
lines_suisse=gpd.read_file('Switzerland/linie-mit-polygon.geojson')

In [ ]:
lines_suisse.head()

# 5.2 the stations

In [ ]:
final_df=pd.read_csv('Switzerland/Suisse_stations.csv')

In [ ]:
final_df.head()

# 5.3 Visualisation 

In [ ]:
#We need a function to convert swiss coordinates into standard coordinates
from pyproj import Proj, transform

def convert(x,y):
    inProj = Proj(init='epsg:2056')
    outProj = Proj(init='epsg:4326')
    x1,y1 = x,y
    x2,y2 = transform(inProj,outProj,x1,y1)
    return [y2,x2]

In [ ]:
for i in range(len(final_df)):
    coordinates = [final_df['Coord. E'][i], final_df['Coord. N'][i]]
    station = final_df['Nom (ordre alphab.)'][i]
    #dailyVisitors = largestStations['Germany'][station][0]
    trainMap.add_child(folium.Marker(location = coordinates, popup=station))

In [ ]:
for line in lines_suisse.geometry:
    lineTuple = []
    lineTupleInvert = list(line.coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap

# 6. An European visualisation

# 6.1 The database


I use the database given by Geonode on https://geonode.wfp.org/layers/geonode:wld_trs_railways_wfp/metadata_detail

In [ ]:
data=pd.read_csv('World/wld_trs_railways_wfp.csv')

I only use data about the countries studied before.

In [ ]:


France=data[data['country']=='France']
Belgium=data[data['country']=='Belgium']
Austria=data[data['country']=='Austria']
Germany=data[data['country']=='Germany']
Suisse=data[data['country']=='Switzerland']

Europe=pd.concat([France,Belgium,Austria,Germany,Suisse])

from shapely import wkt
Europe['shape'] = Europe['shape'].apply(wkt.loads)


geodata=gpd.GeoSeries(Europe['shape'],crs={'init':'epsg:4326'})


# 6.2 Visualisation

I choose the main cities of our countries in a dataframe you can find on https://www.clearlyandsimply.com/clearly_and_simply/2010/10/geocoding-databases-for-europe.html

In [ ]:
Europetown=pd.read_csv('World/european_cities_us_standard.csv')

Cities=['Paris','Marseille 01','Bern','Lyon 01','Linz','Mnchen','Zrich','Charleroi','Basel','Bruxelles','Berlin','Wien','Antwerpen']
        
Europetown1=Europetown[Europetown['European City']=='Lille']        

for city in Cities:
    Europetown1=Europetown1.append(Europetown[Europetown['European City']==city])

Europetown1

In [ ]:
import folium
print(folium.__version__)


europe= folium.Map(location=[45, 5],
                   tiles='OpenStreetMap', zoom_start=5)

for i in range(0,len(Europetown1)):
      europe.add_child(folium.Marker(location=(Europetown1['Latitude'].iloc[i],Europetown1['Longitude'].iloc[i]), popup=Europetown1['European City'].iloc[i]))
#map_suisse
europe.choropleth(geo_data=geodata)
europe